In [1]:
import os

In [2]:
%pwd

'c:\\Users\\tusha\\OneDrive\\Desktop\\Data-Science-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\tusha\\OneDrive\\Desktop\\Data-Science-Project'

In [34]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    input_file: Path

In [35]:
from src.datascience.constant import *
from src.datascience.utils.comman import read_yaml, create_directories

In [36]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: str = CONFIG_FILE_PATH,  # Ensure these variables are defined
                 params_filepath: str = PARAMS_FILE_PATH,
                 schema_filepath: str = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config["artifacts_root"]])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config["data_transformation"]
        create_directories([config["root_dir"]])  # Ensure the correct key format
        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config["root_dir"]),
            input_file=Path(config["input_file"])  # FIXED key reference
        )
        return data_transformation_config


In [37]:
import os
from src.datascience import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [38]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    ## Note we can add diff data transformation techniques such as scaler, PCA, etc.
    def train_test_splitting(self):
        data = pd.read_csv(self.config.input_file)  # FIXED attribute name

        # Splitting the data
        train, test = train_test_split(data, test_size=0.2, random_state=42)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)  # FIXED os.Path.join
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Train and test data split successfully")
        logger.info(f"Train shape: {train.shape}")
        logger.info(f"Test shape: {test.shape}")

        print(train.shape)
        print(test.shape)

In [40]:
# Run the pipeline
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    logger.error(f"Failed to run data transformation: {e}")
    raise e

[2025-02-13 18:10:51,271: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2025-02-13 18:10:51,275: INFO: comman: yaml file: params.yaml loaded successfully]
[2025-02-13 18:10:51,283: INFO: comman: yaml file: schema.yaml loaded successfully]
[2025-02-13 18:10:51,290: INFO: comman: created directory at: artifacts]
[2025-02-13 18:10:51,296: INFO: comman: created directory at: artifacts/data_transformation]


[2025-02-13 18:10:51,634: INFO: 3993229689: Train and test data split successfully]
[2025-02-13 18:10:51,637: INFO: 3993229689: Train shape: (7228, 17)]
[2025-02-13 18:10:51,638: INFO: 3993229689: Test shape: (1807, 17)]
(7228, 17)
(1807, 17)
